In [33]:
import pandas as pd
import pickle
import numpy as np

https://arxiv.org/abs/1604.06737

https://www.fast.ai/2018/04/29/categorical-embeddings/

https://youtu.be/XJ_waZlJU8g?t=4660

https://www.youtube.com/watch?v=5_xFdhfUnvQ&feature=youtu.be

https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

https://github.com/entron/entity-embedding-rossmann

https://github.com/terry-li-hm/rossmann

In [34]:
def apply_cats(df, trn):
    """Changes any columns of strings in df into categorical variables using trn as
    a template for the category codes.
    Parameters:
    -----------
    df: A pandas dataframe. Any columns of strings will be changed to
        categorical values. The category codes are determined by trn.
    trn: A pandas dataframe. When creating a category for df, it looks up the
        what the category's code were in trn and makes those the category codes
        for df.
    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    note the type of col2 is string
    >>> train_cats(df)
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a
    now the type of col2 is category {a : 1, b : 2}
    >>> df2 = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['b', 'a', 'a']})
    >>> apply_cats(df2, df)
           col1 col2
        0     1    b
        1     2    a
        2     3    a
    now the type of col is category {a : 1, b : 2}
    """
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = c.astype('category').cat.as_ordered()
            df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)

In [35]:
PATH='dataset/rossmann/'

In [36]:
# ! pip install pyarrow

In [37]:
joined = pd.read_feather(f'{PATH}joined.feather')
joined_test = pd.read_feather(f'{PATH}joined_test.feather')

Como elegir Validación:

https://www.fast.ai/2017/11/13/validation-sets/

In [38]:
joined.head().T

,0,1,2,3,4
index,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
...,...,...,...,...,...
StateHoliday_bw,0.0,0.0,0.0,0.0,0.0
Promo_bw,5.0,5.0,5.0,5.0,5.0
SchoolHoliday_fw,7.0,1.0,5.0,1.0,1.0
StateHoliday_fw,0.0,0.0,0.0,0.0,0.0


# cat_vars y contin_vars

In [39]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

n = len(joined);

In [40]:
# Me quedo solo con las variables de interes
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

In [41]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

# se define cat_sz

In [42]:
# el +1 es para las unknown
cat_sz = [(c, len(joined[c].cat.categories)+1, min(50, (len(joined[c].cat.categories)+1+1)//2)) for c in cat_vars]

In [43]:
cat_sz

[('Store', 1116, 50),
 ('DayOfWeek', 8, 4),
 ('Year', 4, 2),
 ('Month', 13, 7),
 ('Day', 32, 16),
 ('StateHoliday', 3, 2),
 ('CompetitionMonthsOpen', 26, 13),
 ('Promo2Weeks', 27, 14),
 ('StoreType', 5, 3),
 ('Assortment', 4, 2),
 ('PromoInterval', 4, 2),
 ('CompetitionOpenSinceYear', 24, 12),
 ('Promo2SinceYear', 9, 5),
 ('State', 13, 7),
 ('Week', 53, 27),
 ('Events', 22, 11),
 ('Promo_fw', 7, 4),
 ('Promo_bw', 7, 4),
 ('StateHoliday_fw', 4, 2),
 ('StateHoliday_bw', 4, 2),
 ('SchoolHoliday_fw', 9, 5),
 ('SchoolHoliday_bw', 9, 5)]

In [44]:
# Aplica la misma regla de encoding usada en joined a joined_test
apply_cats(joined_test, joined)

/var/folders/zf/fg68pxbn0ys0ljbx0b3pmxrw0000gq/T/ipykernel_73021/2225282543.py:38: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)
/var/folders/zf/fg68pxbn0ys0ljbx0b3pmxrw0000gq/T/ipykernel_73021/2225282543.py:38: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)
/var/folders/zf/fg68pxbn0ys0ljbx0b3pmxrw0000gq/T/ipykernel_73021/2225282543.py:38: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Catego

In [45]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [46]:
joined = joined.set_index("Date")
joined_test = joined_test.set_index("Date")

In [47]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
# from keras.layers import merge
from keras.optimizers import Adam

In [48]:
from sklearn_pandas import DataFrameMapper
# from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler

In [49]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [50]:
joined['PromoInterval'].astype('str')

Date
2015-07-31                nan
2015-07-31    Jan,Apr,Jul,Oct
2015-07-31    Jan,Apr,Jul,Oct
2015-07-31                nan
2015-07-31                nan
                   ...       
2013-01-01                nan
2013-01-01                nan
2013-01-01    Jan,Apr,Jul,Oct
2013-01-01                nan
2013-01-01                nan
Name: PromoInterval, Length: 844338, dtype: object

In [52]:
cat_mapper = DataFrameMapper(cat_maps)
cat_map_fit = cat_mapper.fit(joined)
cat_cols = len(cat_map_fit.features)
cat_cols

22

In [22]:
# from keras.layers import Flatten, Embedding, Input, merge, Dense, Dropout
from keras.layers import Flatten, Embedding, Input, Dense, Dropout
from keras.models import Model

In [23]:
from keras.layers import Concatenate

     from keras.layers import Concatenate


3. Update deprecated code:
- The code uses the deprecated merge function from keras.layers. Replace it with Concatenate:

In [24]:
cont_out = len(contin_vars)

In [53]:
def get_emb(cat_sz):
    name, c, c2 = cat_sz
    inp = Input((1,), dtype='int64', name=name+'_in')
    u = Flatten(name=name+'_flt')(Embedding(c, c2, input_length=1)(inp))
    return inp, u

def get_contin(name):
    inp = Input((1,), name=name+'_in')
    return inp, Dense(1, name=name+'_d')(inp)



In [62]:
from keras.layers import concatenate

In [79]:
from keras.layers import Input, Embedding, concatenate, Dropout, Dense
from keras.models import Model

def get_model(cat_sz, contin_vars):
    conts = [get_contin(feat) for feat in contin_vars]
    cont_out = [d for inp,d in conts]
    cont_inp = [inp for inp,d in conts]

    embs = [get_emb(feat) for feat in cat_sz]

    x = concatenate([emb for inp,emb in embs] + cont_out)

    x = Dropout(0.02)(x)
    x = Dense(1000, activation='relu', kernel_initializer='uniform')(x)
    x = Dense(500, activation='relu', kernel_initializer='uniform')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[inp for inp,emb in embs] + cont_inp, outputs=x)
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model


In [80]:
model = get_model(cat_sz=cat_sz, contin_vars=contin_vars)

In [82]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Store_in (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 DayOfWeek_in (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 Year_in (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 Month_in (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [83]:
joined.shape

(844338, 39)

In [84]:
joined.columns

Index(['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday',
       'CompetitionMonthsOpen', 'Promo2Weeks', 'StoreType', 'Assortment',
       'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State',
       'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw',
       'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw',
       'CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend',
       'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday', 'Promo',
       'SchoolHoliday', 'Sales'],
      dtype='object')

In [85]:
len(model.input)

38

In [86]:
X_train = joined[['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday',
       'CompetitionMonthsOpen', 'Promo2Weeks', 'StoreType', 'Assortment',
       'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State',
       'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw',
       'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw',
       'CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend',
       'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday', 'Promo',
       'SchoolHoliday']]
y_train = joined[[dep]]

In [100]:
X_train.shape

(844338, 38)

In [119]:
# X_train_list = np.hsplit(X_train.values, X_train.shape[1])
X_train_list = np.vsplit(X_train.values, X_train.shape[0])
X_train_list = np.vsplit(float(X_train.values), X_train.shape[0])

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
X_train_list

[array([[1, 5, 2015, 7, 31, False, 24, 0, 'c', 'a', nan, 2008, 1900, 'HE',
         31, 'Fog', 5.0, 5.0, 0.0, 0.0, 7.0, 5.0, 1270.0, 23.0, 16.0, 8.0,
         98.0, 54.0, 18.0, 24.0, 11.0, 1.0, 85.0, 83.0, 57.0, 0.0, 1.0,
         1.0]], dtype=object),
 array([[2, 5, 2015, 7, 31, False, 24, 25, 'a', 'a', 'Jan,Apr,Jul,Oct',
         2007, 2010, 'TH', 31, 'Fog', 1.0, 5.0, 0.0, 0.0, 1.0, 5.0, 570.0,
         19.0, 13.0, 7.0, 100.0, 62.0, 25.0, 14.0, 11.0, 4.0, 80.0, 83.0,
         67.0, 0.0, 1.0, 1.0]], dtype=object),
 array([[3, 5, 2015, 7, 31, False, 24, 25, 'a', 'a', 'Jan,Apr,Jul,Oct',
         2006, 2011, 'NW', 31, 'Fog', 5.0, 5.0, 0.0, 0.0, 5.0, 5.0,
         14130.0, 21.0, 13.0, 6.0, 100.0, 61.0, 24.0, 14.0, 5.0, 2.0,
         86.0, 83.0, 57.0, 0.0, 1.0, 1.0]], dtype=object),
 array([[4, 5, 2015, 7, 31, False, 24, 0, 'c', 'c', nan, 2009, 1900, 'BE',
         31, nan, 1.0, 5.0, 0.0, 0.0, 1.0, 5.0, 620.0, 19.0, 14.0, 9.0,
         94.0, 61.0, 30.0, 23.0, 16.0, 6.0, 74.0, 83.0, 67.0, 0

In [106]:
len(X_train_list)

844338

In [109]:
model.fit(X_train_list, y_train.values[:,0])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [115]:
from keras import backend as K

In [116]:
import numpy as np

# Convert input data to NumPy arrays if not already
X_train_array = K.constant( np.asarray(X_train_list) )
y_train_array = K.constant( np.asarray(y_train.values[:, 0]) )

# ... Rest of your code ...

# Assuming your model is already defined using the get_model function
model = get_model(cat_sz, contin_vars)

# Ensure that input shapes match
assert X_train_array.shape[0] == y_train_array.shape[0]

# Train the model
model.fit(X_train_array, y_train_array)


ValueError: could not convert string to float: 'c'

In [91]:
for d in X_train_list:
    print(d)

[[1]
 [2]
 [3]
 ...
 [769]
 [948]
 [1097]]
[[5]
 [5]
 [5]
 ...
 [2]
 [2]
 [2]]
[[2015]
 [2015]
 [2015]
 ...
 [2013]
 [2013]
 [2013]]
[[7]
 [7]
 [7]
 ...
 [1]
 [1]
 [1]]
[[31]
 [31]
 [31]
 ...
 [1]
 [1]
 [1]]
[[False]
 [False]
 [False]
 ...
 [True]
 [True]
 [True]]
[[24]
 [24]
 [24]
 ...
 [0]
 [0]
 [24]]
[[0]
 [25]
 [25]
 ...
 [5]
 [0]
 [0]]
[['c']
 ['a']
 ['a']
 ...
 ['b']
 ['b']
 ['b']]
[['a']
 ['a']
 ['a']
 ...
 ['b']
 ['b']
 ['b']]
[[nan]
 ['Jan,Apr,Jul,Oct']
 ['Jan,Apr,Jul,Oct']
 ...
 ['Jan,Apr,Jul,Oct']
 [nan]
 [nan]]
[[2008]
 [2007]
 [2006]
 ...
 [1900]
 [1900]
 [2002]]
[[1900]
 [2010]
 [2011]
 ...
 [2012]
 [1900]
 [1900]]
[['HE']
 ['TH']
 ['NW']
 ...
 ['NW']
 ['BW']
 ['RP']]
[[31]
 [31]
 [31]
 ...
 [1]
 [1]
 [1]]
[['Fog']
 ['Fog']
 ['Fog']
 ...
 ['Rain']
 ['Fog-Rain']
 ['Rain']]
[[5.0]
 [1.0]
 [5.0]
 ...
 [1.0]
 [1.0]
 [1.0]]
[[5.0]
 [5.0]
 [5.0]
 ...
 [0.0]
 [0.0]
 [0.0]]
[[0.0]
 [0.0]
 [0.0]
 ...
 [1.0]
 [2.0]
 [1.0]]
[[0.0]
 [0.0]
 [0.0]
 ...
 [1.0]
 [1.0]
 [1.0]]
[[7.0]
 [1.